In [1]:
from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import PlainTextResponse
from pydantic import BaseModel
from typing import Dict, Any, Union, List, Optional
import inspect
import uvicorn
import nest_asyncio
import socket
import pandas as pd
import json
from threading import Thread


# Import from Rules.ipynb
%run Rules.ipynb

nest_asyncio.apply()

app = FastAPI(title="Business Rules Engine")
app.add_middleware(
    CORSMiddleware,
    allow_origins = ["*"],
    allow_credentials = True,
    allow_methods = ["*"],
    allow_headers = ["*"]
)

# ---------- Data Functions ----------
DATA_FUNCTIONS = {
    "get_customers": get_customers,
    "get_accounts": get_accounts,
    "get_transactions": get_transactions,
    "multi_join_cust_acc": multi_join_cust_acc,
    "multi_join_cust_tran": multi_join_cust_tran,
    "multi_join_acc_tran": multi_join_acc_tran,
    "super_join_cust_acc_tran": super_join_cust_acc_tran
}

# ---------- Rule Management ----------
RULE_DESCRIPTIONS = {
    f"rule_{i:02d}": func.__doc__ or f"Rule {i} description"
    for i, func in enumerate(RULE_REGISTRY.values(), 1)
}

TEMP_RULES = {}

class RuleModel(BaseModel):
    rule_id: str
    description: str
    function_name: str
    column: str
    condition: str
    value: Any
    code: str
    extra: Dict[str, Any] = {}

@app.get("/rules", response_class=PlainTextResponse)
def list_rules():
    predefined_rules = {
        "rule_01": "Customers without any associated account",
        "rule_02": "Accounts with no transaction history",
        "rule_03": "Transactions above ₹1,00,000",
        "rule_04": "Accounts activated before 2020",
        "rule_05": "Transactions with zero amount",
        "rule_06": "Customers with more than one account",
        "rule_07": "Accounts inactive for more than 1 year",
        "rule_08": "Transactions on non-active accounts",
        "rule_09": "Customers missing city details",
        "rule_10": "Transactions that occurred before account activation",
        "rule_11": "5+ transactions within a 1-minute window",
        "rule_12": "10+ small (< ₹100) transactions per account",
        "rule_13": "Customers transacting across multiple accounts",
        "rule_14": "Invalid rows with mismatched customer IDs",
        "rule_15": "Customers with duplicate names"
    }
    lines = ["📋 Business Rules:\n"]
    for rule_id, desc in predefined_rules.items():
        lines.append(f"🔹 {rule_id}: {desc} (Predefined)")
    for rule_id, rule_data in TEMP_RULES.items():
        desc = rule_data.get("description", "Dynamic rule")
        lines.append(f"🆕 {rule_id}: {desc} (Dynamic)")
    return "\n".join(lines)

import numpy as np

@app.get("/run_rule/{rule_id}")
def run_rule_by_id(rule_id: str):
    if rule_id in RULE_REGISTRY:
        try:
            result = run_rule(rule_id)
            return {
                "rule_id": rule_id,
                "description": RULE_DESCRIPTIONS.get(rule_id, ""),
                "result": result
            }
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Error running rule: {e}")
    if rule_id in TEMP_RULES:
        rule = TEMP_RULES[rule_id]
        fn = DATA_FUNCTIONS[rule["function_name"]]
        df = fn()
        local_vars = {"df": df.copy()}
        try:
            exec(rule["code"], {}, local_vars)
            result_df = local_vars.get("df", df)
            result_json = json.loads(result_df.replace({np.nan: None}).to_json(orient="records"))
            return {
                "rule_id": rule_id,
                "description": rule["description"],
                "result": result_json
            }
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Dynamic rule error: {e}")
    raise HTTPException(status_code=404, detail="Rule not found")


@app.post("/add_rule")
def add_dynamic_rule(rule: RuleModel):
    if rule.rule_id in RULE_DESCRIPTIONS or rule.rule_id in TEMP_RULES:
        raise HTTPException(status_code=400, detail="Rule ID already exists")
    if rule.function_name not in DATA_FUNCTIONS:
        raise HTTPException(status_code=400, detail="Function not found")
    TEMP_RULES[rule.rule_id] = rule.dict()
    return {"message": "Rule added successfully"}

@app.get("/functions", response_class=PlainTextResponse)
def list_functions(level: int = Query(0, ge=0, le=2), function_name: Optional[str] = Query(None)):
    if level == 0:
        description = """
🟢 Level 0 - Basic Data Retrieval:
These are the foundational functions that fetch raw, unjoined data from the database. They are typically used as a base for analysis or filtering before applying any business logic or joining operations.
They offer complete flexibility with multiple filtering options.

Functions:
- get_customers
- get_accounts
- get_transactions
"""
        if function_name == "get_customers":
            description += """
get_customers:
- customer_id: ID(s) of customers to retrieve.
- name: Exact or partial match for customer name.
- city: Filter by city name(s).
- update_date: Filter specific dates (YYYY-MM-DD).
- exact_match: False allows partial matching.
- min_update_date / max_update_date: Filter by a range of update dates.
"""
        elif function_name == "get_accounts":
            description += """
get_accounts:
- account_no: One or more account numbers.
- account_type: Account type (e.g., savings, current).
- customer_id: Filter by customer IDs.
- account_status: e.g., active, closed.
- activation_date: Specific dates.
- exact_match: Set to False for partial search.
- min_activation_date / max_activation_date: Filter by range.
"""
        elif function_name == "get_transactions":
            description += """
get_transactions:
- transaction_id: Filter by transaction ID(s).
- account_no: Account numbers involved.
- customer_id: Customer IDs involved.
- amount: Exact value(s).
- min_amount / max_amount: Range filters.
- transaction_time: Timestamps (exact or list).
- min_transaction_time / max_transaction_time: Time window filter.
"""
        return description

    elif level == 1:
        description = """
🟡 Level 1 - Joined Data Retrieval:
These functions combine two related entities such as customers and accounts, or accounts and transactions. They're commonly used for identifying relationships or cross-checking data between two tables.
Ideal for use cases where a simple join operation is needed to understand dependencies or relationships in the data.

Functions:
- multi_join_cust_acc
- multi_join_cust_tran
- multi_join_acc_tran
"""
        if function_name == "multi_join_cust_acc":
            description += """
multi_join_cust_acc:
Joins customer and account data on specified keys.

Parameters:
- join_type (str): Type of join (e.g., 'left', 'inner'). Default is 'left'.
- customer_on (str or List[str]): Column(s) in customer data to join on.
- account_on (str or List[str]): Column(s) in account data to join on.
- customer_filters (dict): Filters to apply if cust DataFrame is not provided.
- account_filters (dict): Filters to apply if acc DataFrame is not provided.
- cust (pd.DataFrame): Optional. If provided, used directly as customers data.
- acc (pd.DataFrame): Optional. If provided, used directly as accounts data.
"""
        elif function_name == "multi_join_cust_tran":
            description += """
multi_join_cust_tran:
Joins customer and transaction data on specified keys.

Parameters:
- join_type (str): Type of SQL join ('left', 'inner', etc.). Default is 'left'.
- customer_on (str or List[str]): Key(s) from the customer table to join on.
- transaction_on (str or List[str]): Key(s) from the transaction table to join on.
- customer_filters (dict): Filters for get_customers if no cust DataFrame is passed.
- transaction_filters (dict): Filters for get_transactions if no tran DataFrame is passed.
- cust (pd.DataFrame): Optional. Pre-filtered customer DataFrame.
- tran (pd.DataFrame): Optional. Pre-filtered transaction DataFrame.
"""
        elif function_name == "multi_join_acc_tran":
            description += """
multi_join_acc_tran:
Joins account and transaction data on specified keys.

Parameters:
- join_type (str): Type of join ('left', 'inner', etc.). Default is 'left'.
- account_on (str or List[str]): Key(s) from the account table.
- transaction_on (str or List[str]): Key(s) from the transaction table.
- account_filters (dict): Filters to apply on get_accounts() if acc not provided.
- transaction_filters (dict): Filters to apply on get_transactions() if tran not provided.
- acc (pd.DataFrame): Optional. Pre-filtered account DataFrame.
- tran (pd.DataFrame): Optional. Pre-filtered transaction DataFrame.
"""
        return description

    elif level == 2:
        return """
🔴 Level 2 -Super Function:
This advanced utility joins all three core tables: customers, accounts, and transactions. It allows detailed correlation and traceability across the entire customer lifecycle. This is highly useful for detecting anomalies, customer behavior analytics, and policy compliance.

Function://
- super_join_cust_acc_tran

super_join_cust_acc_tran:
Parameters:
- join_type (str): Type of join to apply ('left', 'inner', etc.)
- customer_filters (dict): Filters for customers if 'cust' not provided.
- account_filters (dict): Filters for accounts if 'acc' not provided.
- transaction_filters (dict): Filters for transactions if 'tran' not provided.
- customer_account_key (str or List[str]): Join key(s) between customers and accounts.
- account_transaction_key (str or List[str]): Join key(s) between accounts and transactions.
- cust (pd.DataFrame): Optional. Pre-filtered customers DataFrame.
- acc (pd.DataFrame): Optional. Pre-filtered accounts DataFrame.
- tran (pd.DataFrame): Optional. Pre-filtered transactions DataFrame.
"""

    return "Invalid level. Use level 0, 1, or 2."

class FunctionCall(BaseModel):
    function_name: str
    parameters: Dict[str, Any] = {}

@app.post("/run_function")
def run_function_with_parameters(body: FunctionCall):
    fn = DATA_FUNCTIONS.get(body.function_name)
    if not fn:
        raise HTTPException(status_code=404, detail="Function not found")
    try:
        result = fn(**body.parameters)
        if hasattr(result, 'to_dict'):
            result_json = json.loads(result.replace({np.nan: None}).to_json(orient="records"))
            return result_json
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



@app.delete("/delete_rule/{rule_id}")
def delete_rule(rule_id: str):
    if rule_id not in TEMP_RULES:
        raise HTTPException(status_code=404, detail="Rule ID not found")
    del TEMP_RULES[rule_id]
    return {"message": f"Rule '{rule_id}' deleted successfully"}

# ---------- Launch FastAPI Server ----------
def find_open_port(start=8000):
    for port in range(start, 8100):
        with socket.socket() as s:
            if s.connect_ex(("127.0.0.1", port)) != 0:
                return port
    raise RuntimeError("No open ports")

def start_server():
    port = find_open_port()
    print(f"\n✅ FastAPI running: http://127.0.0.1:{port}/docs")
    uvicorn.run(app, host="127.0.0.1", port=port)

Thread(target=start_server, daemon=True).start()